In [94]:
import csv
import pandas as pd
import re
import numpy as np

df_Medica = pd.read_csv("archives_medecine_navale.tsv",sep='\t',low_memory=False)
df_Medica

,cote,refimg,page,pagecle,chapitre,pagimgcle,pagimg,pagimgtxt,pagimgtxtcom
0,90156x1864x01,2,[page blanche],NaN,NaN,NaN,NaN,NaN,NaN
1,90156x1864x01,1,[sans numérotation],O,[Page de faux-titre],NaN,NaN,NaN,NaN
2,90156x1864x01,3,[sans numérotation],O,[Page de titre],NaN,NaN,NaN,NaN
3,90156x1864x01,4,[page blanche],NaN,NaN,NaN,NaN,NaN,NaN
4,90156x1864x01,5,5,O,Introduction,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
66566,90156x1919x107,476,476,O,"Bibliographie. Naval Hygiene, by James Chamber...",NaN,NaN,NaN,NaN
66567,90156x1919x107,477,477,O,Bulletin officiel. / Avril 1919,NaN,NaN,NaN,NaN
66568,90156x1919x107,478,478,O,Table analytique des matières du tome cent sep...,NaN,NaN,NaN,NaN
66569,90156x1919x107,479,479,NaN,Table analytique des matières du tome cent sep...,NaN,NaN,NaN,NaN


In [140]:
#Ne prendre que les débuts d'articles/chapitres
df_Medica_entrees = df_Medica.loc[df_Medica["pagecle"]=="O"]

#Retirer les doublons
df_Medica_entrees =df_Medica_entrees.drop_duplicates(keep='first')

#Extraire les années : 
df_Medica_entrees["revue_annee"]=df_Medica_entrees["cote"].str.split("x").str[1].astype(int)

#Extraire les années : 
df_Medica_entrees["revue_numero"]=df_Medica_entrees["cote"].str.split("x").str[2]
#ATTENTION: type(revue_numero)=str

#Ordonner par années 
df_Medica_entrees=df_Medica_entrees.sort_values(by=['revue_annee'])

#Création de l'article_id
model_article_id = arr = np.dstack((df_Medica_entrees["revue_annee"].values.astype(str), df_Medica_entrees["page"].values.astype(str)))

list_art_id=[]
for l in arr:
    for g in l:
        list_art_id.append("AMN-"+"-".join(g))
df_Medica_entrees["article_id"]=list_art_id


In [142]:
#Réorganisation des colonnes
df_Medica_entrees=df_Medica_entrees[['cote','article_id','chapitre','revue_annee','revue_numero','page']]
df_Medica_entrees

,cote,article_id,chapitre,revue_annee,revue_numero,page
1,90156x1864x01,AMN-1864-[sans numérotation],[Page de faux-titre],1864,01,[sans numérotation]
653,90156x1864x02,AMN-1864-172,Variétés. Hôpitaux de San Francisco (Californi...,1864,02,172
651,90156x1864x02,AMN-1864-170,Variétés. / Hôpitaux de San Francisco (Califor...,1864,02,170
649,90156x1864x02,AMN-1864-168,Variétés. Discours d'adieu de M. le professeur...,1864,02,168
646,90156x1864x02,AMN-1864-165,Bibliographie. Eau Thermo-minérale de la ravin...,1864,02,165
...,...,...,...,...,...,...
49130,90156x1914_1916x102,AMN-19141916-169,Notes et mémoires originaux. Quelques notes po...,19141916,102,169
49131,90156x1914_1916x102,AMN-19141916-170,Notes et mémoires originaux. Quelques notes po...,19141916,102,170
49132,90156x1914_1916x102,AMN-19141916-171,Notes et mémoires originaux. Quelques notes po...,19141916,102,171
49112,90156x1914_1916x102,AMN-19141916-151,Revue analytique. Contribution à l'étude de l'...,19141916,102,151


### Séparation du titre/auteur et de la description:
#### Boucle très longue,pas besoin de le repasser, je l'ai copié dans AMN_temporaire-clean.csv qui est importé à la cellule suivante

In [143]:
#Nouvelle colonne pour l'article-id : 
# df_Medica_entrees["article_description"] = df_Medica_entrees.apply(lambda _:'', axis=1)

# df_Medica_entrees['article_description'] = pd.Series(dtype=str)

# pattern_descr = r'(\, )?(\.{2,})(.*)'

# for entree in df_Medica_entrees.chapitre:
#     i +=1
#     if re.search(pattern_descr,entree):
#         desc = re.search(pattern_descr,entree).group(3)
#         #Ajout valeur à article_description : 
#         df_Medica_entrees.loc[i-1,"article_description"] = re.search(pattern_descr,entree).group(3)
#         #retrait de la description du titre : 
#         df_Medica_entrees.loc[i-1,"chapitre"] = re.sub(pattern_descr,r'\1',entree)

#df_Medica_entrees.to_csv("AMN_temporaire.tsv",sep='\t')

In [163]:
df_Medica_entrees= pd.read_csv("AMN_temporaire-clean.csv",sep="\t")

In [164]:
#Extraction des AMN post-1898
annees_vouluesAMN = list(range(1898,1941))
df_AMN1898_1940=df_Medica_entrees.loc[df_Medica_entrees["revue_annee"].isin(annees_vouluesAMN)]
df_AMN1898_1940=df_AMN1898_1940.sort_values(by=['revue_annee'])

#df_AMN1898_1940

#Extraction des AMN au temps des archives de médecine navale et coloniale
annees_voulues_ArchMedCol = list(range(1890,1898))
df_ArchMedCol=df_Medica_entrees.loc[df_Medica_entrees["revue_annee"].isin(annees_voulues_ArchMedCol)]
df_ArchMedCol=df_ArchMedCol.sort_values(by=['revue_annee'])
#df_ArchMedCol

## DF AMN 1898 à 1940 : 

In [165]:
df_AMN1898_1940 =df_AMN1898_1940.drop_duplicates(subset="chapitre",keep='first') #divisé par4 ! 
df_AMN1898_1940.to_csv("./AMN_1898-1940.tsv",sep="\t")
df_AMN1898_1940

,Unnamed: 0,cote,article_id,chapitre,revue_annee,page,article_description
13382,33549,90156x1898x69,AMN-1898-[sans numérotation],[Page de faux-titre],1898,[sans numérotation],NaN
13679,34194,90156x1898x70,AMN-1898-163,"Colonne expéditionnaire dans le Haut-Dahomey, ...",1898,163,NaN
13678,34192,90156x1898x70,AMN-1898-161,"Colonne expéditionnaire dans le Haut-Dahomey, ...",1898,161,NaN
13677,34187,90156x1898x70,AMN-1898-156,Livres reçus. / Bulletin officiel. / Juillet 1...,1898,156,NaN
13676,34186,90156x1898x70,AMN-1898-155,"Bibliographie. La culture des mers en Europe, ...",1898,155,NaN
...,...,...,...,...,...,...,...
27705,65103,90156x1940x130,AMN-1940-32,I. Mémoires originaux. La prophylaxie anti-vén...,1940,32,NaN
27704,65102,90156x1940x130,AMN-1940-31,I. Mémoires originaux. La prophylaxie anti-vén...,1940,31,NaN
27703,65101,90156x1940x130,AMN-1940-30,I. Mémoires originaux. Sur le traitement chiru...,1940,30,NaN
27709,65107,90156x1940x130,AMN-1940-36,I. Mémoires originaux. La prophylaxie anti-vén...,1940,36,NaN


In [167]:
#SECTION

#Nouvelle colonne pour la section de l'article : 
# df_AMN1898_1940['article_section'] = pd.Series(dtype=str)

Liste_sections = [r"Bibliographie",
                  r"Page de faux-titre",
                  r"Page de titre",
                  r"Livres reçus",
                  r"Bulletin officiel",
                  r"Circulaire",
                  r"Loi",
                  r"Instruction",
                  r"Règlement", 
                  r"Conférence",
                  r"Arrêté",
                  r"Mémoires originaux",
                  r"Chronique documentaire",
                 r"Table analytique",
                 r"Statistique médicale",
                 r"Répartition des archives de médecine navale"
                 r"Liste des périodiques échangés"]

print(len(df_AMN1898_1940))

for i in range(len(df_AMN1898_1940)):
    entree = df_AMN1898_1940.iloc[i,3]
    for section in Liste_sections:
# #         print(section)
        if re.search(section,entree):
            df_AMN1898_1940.iloc[i,7] = section
        else :
            df_AMN1898_1940.iloc[i,7] = "article"

14246


In [168]:
df_AMN1898_1940

,Unnamed: 0,cote,article_id,chapitre,revue_annee,page,article_description,article_section
13382,33549,90156x1898x69,AMN-1898-[sans numérotation],[Page de faux-titre],1898,[sans numérotation],NaN,article
13679,34194,90156x1898x70,AMN-1898-163,"Colonne expéditionnaire dans le Haut-Dahomey, ...",1898,163,NaN,article
13678,34192,90156x1898x70,AMN-1898-161,"Colonne expéditionnaire dans le Haut-Dahomey, ...",1898,161,NaN,article
13677,34187,90156x1898x70,AMN-1898-156,Livres reçus. / Bulletin officiel. / Juillet 1...,1898,156,NaN,article
13676,34186,90156x1898x70,AMN-1898-155,"Bibliographie. La culture des mers en Europe, ...",1898,155,NaN,article
...,...,...,...,...,...,...,...,...
27705,65103,90156x1940x130,AMN-1940-32,I. Mémoires originaux. La prophylaxie anti-vén...,1940,32,NaN,article
27704,65102,90156x1940x130,AMN-1940-31,I. Mémoires originaux. La prophylaxie anti-vén...,1940,31,NaN,article
27703,65101,90156x1940x130,AMN-1940-30,I. Mémoires originaux. Sur le traitement chiru...,1940,30,NaN,article
27709,65107,90156x1940x130,AMN-1940-36,I. Mémoires originaux. La prophylaxie anti-vén...,1940,36,NaN,article


In [169]:
#ne conserver quelesarticles:
df_AMN1898_1940articles = df_AMN1898_1940.loc[df_AMN1898_1940['article_section'] == "article"]
df_AMN1898_1940articles

,Unnamed: 0,cote,article_id,chapitre,revue_annee,page,article_description,article_section
13382,33549,90156x1898x69,AMN-1898-[sans numérotation],[Page de faux-titre],1898,[sans numérotation],NaN,article
13679,34194,90156x1898x70,AMN-1898-163,"Colonne expéditionnaire dans le Haut-Dahomey, ...",1898,163,NaN,article
13678,34192,90156x1898x70,AMN-1898-161,"Colonne expéditionnaire dans le Haut-Dahomey, ...",1898,161,NaN,article
13677,34187,90156x1898x70,AMN-1898-156,Livres reçus. / Bulletin officiel. / Juillet 1...,1898,156,NaN,article
13676,34186,90156x1898x70,AMN-1898-155,"Bibliographie. La culture des mers en Europe, ...",1898,155,NaN,article
...,...,...,...,...,...,...,...,...
27705,65103,90156x1940x130,AMN-1940-32,I. Mémoires originaux. La prophylaxie anti-vén...,1940,32,NaN,article
27704,65102,90156x1940x130,AMN-1940-31,I. Mémoires originaux. La prophylaxie anti-vén...,1940,31,NaN,article
27703,65101,90156x1940x130,AMN-1940-30,I. Mémoires originaux. Sur le traitement chiru...,1940,30,NaN,article
27709,65107,90156x1940x130,AMN-1940-36,I. Mémoires originaux. La prophylaxie anti-vén...,1940,36,NaN,article


In [ ]:
df_Medica_articles.to_csv("AMN_articles-1898-1908.tsv",sep="\t")

In [ ]:
df_Medica_98_09_cleaner = pd.read_csv("AMN_articles-1898-1908.tsv",sep="\t")
df_Medica_98_09_cleaner.drop(columns=["Unnamed: 0","Unnamed: 0.1"])

In [ ]:
df_Medica_98_09_cleaner.drop_duplicates(subset="chapitre",keep='first')

In [ ]:
df_Medica_98_09_cleaner.to_csv("./Clean/AMN-articles-98-08_clean.tsv",sep="\t")